In [1]:
import pandas as pd

In [2]:
# imports
job_postings = pd.read_csv('job_postings.csv', engine="python")
companies = pd.read_csv('companies.csv')
company_industries = pd.read_csv('company_industries.csv')
employee_counts = pd.read_csv('employee_counts.csv')
industries = pd.read_csv('industries.csv')
job_industries = pd.read_csv('job_industries.csv')
job_skills = pd.read_csv('job_skills.csv')
salaries = pd.read_csv('salaries.csv')
skills = pd.read_csv('skills.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'job_postings.csv'

In [ ]:
# merge datasets
jobs = pd.merge(job_postings, job_industries, how='left', on='job_id')
#print("merge w job_postings and job_industries: ", jobs.shape[0])
jobs = jobs.merge(industries, how='left', on='industry_id')
#print("merge w industries: ", jobs.shape[0])
jobs = jobs.merge(job_skills, how='left', on='job_id')
#print("merge w job_skills: ", jobs.shape[0])
jobs = jobs.merge(skills, how='left', on='skill_abr')
#print("merge w skills: ", jobs.shape[0])
jobs = jobs.merge(salaries, how='left', on='job_id')
#print("merge w salaries: ", jobs.shape[0])
jobs = jobs.merge(companies, how='left', on='company_id')
#print("merge w companies: ", jobs.shape[0])
jobs = jobs.merge(company_industries, how='left', on='company_id')
#print("merge w company_industries: ", jobs.shape[0])
jobs = jobs.merge(employee_counts, how='left', on='company_id')
#print("merge w employee_counts: ", jobs.shape[0])

In [ ]:
# dropping duplicate columns
jobs.drop(columns=['max_salary_y', 'med_salary_y', 'min_salary_y',
                   'pay_period_y', 'currency_y', 'compensation_type_y'], inplace=True)
# dropping irrelevant columns
jobs.drop(columns=['scraped', 'salary_id', 'url', 'time_recorded', 'currency_x', 'posting_domain',
                   'industry_id', 'closed_time', 'listed_time'], inplace=True)
# renaming columns
jobs.rename(columns={'max_salary_x':'max_salary', 'med_salary_x':'med_salary', 'min_salary_x':'min_salary',
                     'compensation_type_x':'compensation_type', 'description_x':'job_description',
                     'state':'company_state', 'country':'company_country', 'city':'company_city',
                     'zip_code':'company_zip_code', 'address':'company_address', 'name':'company_name',
                     'industry_name':'job_industry/function', 'industry':'company_industry',
                     'description_y':'company_description', 'pay_period_x':'pay_period'}, inplace=True)

In [3]:
job_postings_reduce_dim = job_postings.drop(['scraped', 'compensation_type', 'skills_desc', 'application_url', 'pay_period', 'job_posting_url'], axis=1)
#job_id(to merge), company_id(to merge), title, description, max_salary, med_salary, min_salary, formatted_work_type, location, applies, original_listed_time,
#remote_allowed,
job_postings_reduce_dim.columns

NameError: name 'job_postings' is not defined

### Overall plot

In [ ]:
jobs['location'] = jobs['location'].replace('New York City Metropolitan Area', 'New York, NY')

job_postings_locations = jobs['location'].value_counts().nlargest(21)

# Exclude entries where 'location' is "United States" or "0"
job_filtered = job_postings_locations
job_filtered = jobs[(jobs['location'] != "United States") & (jobs['location'] != "0")]
job_filtered['location'] = job_filtered['location'].replace('New York City Metropolitan Area', 'New York, NY')

# Continue with the analysis after excluding these entries:
top_cities_filtered = job_filtered['location'].value_counts().nlargest(10).index.tolist()
top_skills_filtered = job_filtered['skill_name'].value_counts().nlargest(5).index.tolist()

# Prepare the Data: Filter the dataframe for these top cities and skills, and count job postings
filtered_df_filtered = job_filtered[job_filtered['location'].isin(top_cities_filtered) & job_filtered['skill_name'].isin(top_skills_filtered)]
aggregated_df_filtered = filtered_df_filtered.groupby(['location', 'skill_name']).size().reset_index(name='job_postings_num')

aggregated_df_filtered

import altair as alt

chart = alt.Chart(aggregated_df_filtered).mark_bar().encode(
    x=alt.X('location', axis=alt.Axis(title='Location'), sort = alt.EncodingSortField(field = 'job_postings_num', op='sum', order = 'descending')),
    y=alt.Y('job_postings_num:Q', axis=alt.Axis(title='Number of Job Postings')),
    color='skill_name:N',
    tooltip=['location', 'skill_name', 'job_postings_num']
).properties(
    title='Job Postings by City, State and Skill',
    width=600,
    height=400
).interactive()

chart
chart.save("overall.html")

### Word cloud plot

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Extract job titles column
job_titles = jobs['title']

# Join all job titles into a single string
all_titles_text = ' '.join(job_titles)

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_titles_text)

# Display the word cloud
plt.figure(figsize=(20, 12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### Heatmap plot

In [ ]:
import seaborn as sns

# Assuming 'skills' and 'experience_level' are the relevant columns in your DataFrame
# Group by 'experience_level' and 'skills', then count the frequency
aggregated_df = jobs.groupby(['experience_level', 'skill_name']).size().reset_index(name='frequency')

# Pivot the data to get it in the right format for the heatmap
pivot_df = aggregated_df.pivot(index='experience_level', columns='skill_name', values='frequency')

# Select top 10 skills
top_skills = pivot_df.sum().nlargest(10).index
pivot_df = pivot_df[top_skills]

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_df, cmap='Reds', linewidths=.5)
plt.title('Heatmap of Skill Name by Experience Level')
plt.show()

### Median and Average Salary Map plot

In [ ]:
def extract_state(location):
    parts = location.split(',')
    if len(parts) > 1:
        state = parts[-1].strip()
        if state != 'United States':
            return state
    return None

In [ ]:
job_postings_reduce_dim['state'] = job_postings_reduce_dim['location'].apply(extract_state)

job_postings_reduce_dim = job_postings_reduce_dim.dropna(subset=['state'])


In [ ]:
# Dictionary mapping state abbreviations to full state names
state_abbr_to_name = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

# Replace state abbreviations with full state names
job_postings_reduce_dim['state'] = job_postings_reduce_dim['state'].map(state_abbr_to_name)

In [ ]:
job_postings_reduce_dim = job_postings_reduce_dim.dropna(subset=['med_salary'], how='any')

minimum_salary = job_postings_reduce_dim['med_salary'].min()
print("Minimum salary:", minimum_salary)
minimum_salary = job_postings_reduce_dim['med_salary'].median()
print("Median salary:", minimum_salary)
minimum_salary = job_postings_reduce_dim['med_salary'].max()
print("Maximum salary:", minimum_salary)

# Find the top 10 most frequent industries
top_industries = job_postings_reduce_dim['title'].value_counts().head(10).index.tolist()

# Create a DataFrame containing only the top industries
top_industries_df = job_postings_reduce_dim[job_postings_reduce_dim['title'].isin(top_industries)].copy()

num_rows = top_industries_df.shape[0]
print("Number of rows in the DataFrame:", num_rows)

In [ ]:
import geopandas as gpd
import folium
from branca.colormap import LinearColormap

In [ ]:
state_data = gpd.read_file("C:\\Users\\hjlac\\Downloads\\ne_110m_admin_1_states_provinces\\ne_110m_admin_1_states_provinces.shx")
print(state_data.columns)
color_maps = {}

In [ ]:
print(state_data['name'])

print("\njob_postings_reduce_dim 'state' column:")
print(job_postings_reduce_dim['state'].iloc[0])  # Assuming you have a DataFrame named 'job_postings_reduce_dim'

In [ ]:
print("Unique state names in GeoJSON data:", state_data['name'].unique())
print("Unique state names in DataFrame:", job_postings_reduce_dim['state'].unique())
state_data = state_data[state_data['name'] != 'District of Columbia']

In [ ]:
# Get unique state names from GeoJSON data
geojson_states = state_data['name'].unique()

# Get unique state names from DataFrame
data_states = job_postings_reduce_dim['state'].dropna().unique()

# Convert both lists to sets for easier comparison
geojson_states_set = set(geojson_states)
data_states_set = set(data_states)

# Check for discrepancies
discrepancies = geojson_states_set.symmetric_difference(data_states_set)

# Print discrepancies
print("Discrepancies:", discrepancies)

In [ ]:
import folium
from branca.colormap import LinearColormap

# Merge state_data_filtered with job_postings_reduce_dim on state name
merged_data = state_data.merge(job_postings_reduce_dim, how='inner', left_on='name', right_on='state')

# Identify hourly wages under $200
hourly_wages_under_200 = merged_data[merged_data['med_salary'] < 1000]

# Calculate annualized salaries for hourly wages under $200
hours_per_year = 2080  # Assuming 40 hours per week for 52 weeks
hourly_wages_under_200['annual_salary'] = hourly_wages_under_200['med_salary'] * hours_per_year

# Replace original hourly wages with annualized salaries in the merged DataFrame
merged_data.loc[hourly_wages_under_200.index, 'med_salary'] = hourly_wages_under_200['annual_salary']

# Calculate average salaries for each state
average_salaries = merged_data.groupby('name')['med_salary'].mean().reset_index()
average_salaries.columns = ['name', 'average_salary']  # Rename columns for clarity

# Merge average salaries with merged_data
merged_data = merged_data.merge(average_salaries, on='name')

# Create a Folium map object
m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

# Define colormap for median salary
median_salary_cmap = LinearColormap(['magenta', 'cyan', 'blue'], vmin=24000, vmax=140000)

# Create GeoJson layer for median salary
median_salary_layer = folium.GeoJson(
    merged_data,
    name='Median Salary ($)',
    style_function=lambda x: {
        'fillColor': median_salary_cmap(x['properties']['med_salary']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.2,  # Adjust transparency here
    },
    tooltip=folium.GeoJsonTooltip(fields=['name', 'med_salary'], aliases=['State', 'Median Salary'], labels=True, sticky=False),
)

# Define colormap for average salary
average_salary_cmap = LinearColormap(['red', 'yellow', 'green'], vmin=average_salaries['average_salary'].min(), vmax=average_salaries['average_salary'].max())

# Create GeoJson layer for average salary
average_salary_layer = folium.GeoJson(
    merged_data,
    name='Average Salary ($)',
    style_function=lambda x: {
        'fillColor': average_salary_cmap(x['properties']['average_salary']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.05,  # Adjust transparency here
    },
    tooltip=folium.GeoJsonTooltip(fields=['name', 'average_salary'], aliases=['State', 'Average Salary'], labels=True, sticky=False),
)

# Add GeoJson layers to the map
median_salary_layer.add_to(m)
average_salary_layer.add_to(m)

# Add colormaps to the map
median_salary_cmap.caption = 'Median Salary ($)'
average_salary_cmap.caption = 'Average Salary ($)'
median_salary_cmap.add_to(m)
average_salary_cmap.add_to(m)

# Add layer control to toggle between layers
folium.LayerControl(collapsed=False).add_to(m)

# Title the map
title_html = '<h3 align="center" style="font-size:20px"><b>Median and Average Salaries by State</b></h3>'
m.get_root().html.add_child(folium.Element(title_html))

# Save the map as an HTML file
# m.save('interactive_map_median_salary.html')